<a href="https://colab.research.google.com/github/tsam6/Paradigms/blob/main/Class_Accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title initializing data set
!pip3 install mne -q
!git clone https://github.com/N-Nieto/Inner_Speech_Dataset -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.7 MB/s eta 0:00:00


In [ ]:
#@title imports and misc

from google.colab import drive
import mne
import warnings
import numpy as np

from google.colab import drive
drive.mount('/content/gdrive')
from Inner_Speech_Dataset.Python_Processing.Data_extractions import  extract_data_from_subject
from Inner_Speech_Dataset.Python_Processing.Data_processing import  select_time_window, transform_for_classificator, filter_by_condition, filter_by_class

np.random.seed(23)

mne.set_log_level(verbose='warning') #to avoid info at terminal
warnings.filterwarnings(action = "ignore", category = DeprecationWarning )
warnings.filterwarnings(action = "ignore", category = FutureWarning )


Conditions = [["Vis"],["Inner"],["Pron"]]
Intervals = ["action", "think", "entire"]

drive.mount('/gdrive', force_remount=True)

### Hyperparameters

# The root dir has to point to the folder that contains the database
root_dir = "/gdrive/My Drive/inner_speech_set"

# Data Type
datatype = "EEG"

# Sampling rate
fs = 256

# Select the useful par of each trial. Time in seconds
t_start = 0
t_end = 4.5

# Subject number
N_S = 1   #[1 to 10]


Mounted at /content/gdrive
Mounted at /gdrive


In [ ]:
#@title loading subjects

#different subjects used
N_S_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

#Class values (up down left right)
cls_vl = [0, 1, 2, 3]

# sampling rate
fs = 256

# alpha beta and gamma bands frequencies respectively
eeg_bands = [[8,12], [12,30], [32,80] ]

N_S_load = []
N_S_load_Y = []

for N_S in N_S_list:

  X, Y = extract_data_from_subject(root_dir, N_S, datatype)

  X_action = select_time_window(X = X, t_start = .5, t_end = 1, fs = fs)
  X_think = select_time_window(X = X, t_start = 1, t_end = 2, fs = fs)
  X_sig = select_time_window(X = X, t_start = 1, t_end = 4, fs = fs)

  intervals = []

  intervals.append(X_action)
  intervals.append(X_think)
  intervals.append(X_sig)

  N_S_load.append(intervals)
  N_S_load_Y.append(Y)

In [ ]:
#@title Training Code

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn import svm, metrics
from sklearn.neighbors import KNeighborsClassifier

def filter_data(X_val, Y_val, condition):
  # condition based filtering ( visualized, pronounced, inner)
  which_trials = Y_val[:,2]==[condition]

  which_classes = np.logical_or(
  np.logical_or(Y_val[:,1]==cls_vl[0], Y_val[:,1]==cls_vl[1]),
  np.logical_or(Y_val[:,1]==cls_vl[2], Y_val[:,1]==cls_vl[3])
)
  which_trials = np.logical_and(which_trials, which_classes)

  X_trans = X_val[which_trials]
  Y_trans = Y_val[which_trials]

  return X_trans, Y_trans

def val(X_val):

  X_val = X_val[:,:,::2] # gets frequency for every other
  X_tran_erp = X_val[:,eeg,:] # gets values for the nth channel

  return X_tran_erp

def val1(X_val):

  X_tran_psd = X_val[:,eeg,:] #gets values for the nth channel

  return X_tran_psd

def indexer(freqs, psd):
  # index the frequencies so it isolates only the specific bands for necessary data collection
  indexer1 = freqs>eeg_bands[0][0]
  indexer2 = freqs<eeg_bands[0][1]
  indexer_alpha = np.logical_and(indexer1, indexer2)

  indexer1 = freqs>eeg_bands[1][0]
  indexer2 = freqs<eeg_bands[1][1]
  indexer_beta = np.logical_and(indexer1, indexer2)

  indexer1 = freqs>eeg_bands[2][0]
  indexer2 = freqs<eeg_bands[2][1]
  indexer_gamma = np.logical_and(indexer1, indexer2)

  # averages all of the PSDs across the alpha band

  alpha = psd[:,indexer_alpha].mean(axis=-1)
  beta = psd[:,indexer_beta].mean(axis=-1)
  gamma = psd[:,indexer_gamma].mean(axis=-1)
  psd_avg = np.vstack([alpha, beta, gamma]).T

  return psd_avg


def train_evaluate(X_val, Y_val, model):
  # splits data into training and testing sets
  X_train, X_test, Y_train, Y_test = train_test_split(
            X_val, Y_val, test_size=0.1)

  clf = model
  clf.fit(X_train, Y_train)

  Y_pred = clf.predict(X_test)
  avg = metrics.accuracy_score(Y_test, Y_pred)*100
  return avg

def evaluate(X_val, Y_val, model):

  psd_eeg_scores = []
  erp_eeg_scores = []
  for eeg in range(128):

    inr_erp_scores = []
    inr_psd_scores = []
    for inr in range(3):

      con_psd_scores = []
      con_erp_scores = []
      for con in range(3):

        avg_erp = 0
        avg_psd = 0
        values = 10
        for trials in range(10):

          X_trans, Y_trans = filter_data(X_val[inr], Y_val, con)

          X_trans_psd = val1(X_trans)
          X_trans_erp = val(X_trans)

          freqs, psd = signal.welch(X_trans_psd, fs, nperseg=128)

          avg_erp += train_evaluate(X_trans_erp, Y_trans[:,1], model)
          avg_psd += train_evaluate(indexer(freqs, psd), Y_trans[:,1], model)

        con_psd_scores.append(avg_psd/values)
        con_erp_scores.append(avg_erp/values)

      inr_psd_scores.append(avg_psd)
      inr_erp_scores.append(avg_erp)

    psd_eeg_scores.append(inr_psd_scores)
    erp_eeg_scores.append(inr_erp_scores)

  return psd_eeg_scores, erp_eeg_scores



In [ ]:
#@title model function


def model_run(model_name, model):
  globals()[model_name + '_erp_scores'] = []
  globals()[model_name + '_psd_scores'] = []

  for N_S in N_S_list:
      psd_scores, erp_scores = evaluate(N_S_load[N_S - 1], N_S_load_Y[N_S-1], model)
      globals()[model_name + '_erp_scores'].append(erp_scores)
      globals()[model_name + '_psd_scores'].append(psd_scores)
  globals()[model_name + '_erp_score_avg'] = np.mean(globals()[model_name + '_erp_scores'], axis=0)
  globals()[model_name + '_psd_score_avg'] = np.mean(globals()[model_name + '_psd_scores'], axis=0)

In [ ]:
#@title SVC code
model_run('SVC', svm.SVC())

SVC_erp_score_avg.shape

KeyboardInterrupt: 

In [ ]:
#@title SVC code
model_run('SVC', svm.SVC())